In [ ]:
## Installing and importing libraries
%pip install -qU openai==0.28 transformers plotly scikit-learn transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 67.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import openai
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding
from transformers import GPT2TokenizerFast
from tqdm.auto import tqdm
import os

tqdm.pandas()

In [ ]:

import spacy
import numpy as np

# Load spaCy model with GloVe embeddings
import en_core_web_sm

nlp = en_core_web_sm.load()

def custom_embedding(text, model_name="text-embedding-ada-002"):
    # Process the text with spaCy
    doc = nlp(text)

    # Extract word embeddings and average them to get the text embedding
    word_embeddings = [token.vector for token in doc if token.has_vector]

    if not word_embeddings:
        return None  # No embeddings found for any word in the text

    text_embedding = np.mean(word_embeddings, axis=0)

    # Create a response dictionary
    response = {
        "data": [
            {
                "embedding": text_embedding.tolist(),
                "index": 0,
                "object": "embedding"
            }
        ],
        "model": model_name,
        "object": "list",
        "usage": {
            "prompt_tokens": len(text.split()),
            "total_tokens": len(text.split())
        }
    }

    return response

# Example usage
text = "Rome"
response = custom_embedding(text)

if response["data"][0]["embedding"] is not None:
    print(f"Custom Embedding for '{text}': {response['data'][0]['embedding']}")
else:
    print(f"No embeddings found for words in '{text}'.")

print(response)


Custom Embedding for 'Rome': [-1.436251163482666, -0.4631139636039734, -0.2887111008167267, 0.1043136715888977, -0.18004582822322845, 0.3143523633480072, -0.17387962341308594, 0.6757763624191284, -1.3850526809692383, -0.3222927451133728, 1.3471592664718628, 0.33096542954444885, -0.7217952013015747, 0.8920304775238037, -1.20805025100708, 0.46127259731292725, -1.305800199508667, -0.8477014303207397, 0.8280575275421143, 0.2849760949611664, -1.4031113386154175, 0.33332395553588867, -0.6530104875564575, -1.0358444452285767, 0.4907127618789673, 0.2932211756706238, 0.6595003008842468, 2.175729513168335, -0.5897372961044312, 0.6383960247039795, -1.1273974180221558, -0.6744024157524109, 1.082930326461792, 1.0616108179092407, -0.07839717715978622, -1.8823751211166382, -0.028083711862564087, -0.17612075805664062, 0.969205915927887, 0.784468412399292, -1.2386583089828491, -0.0038348734378814697, -1.2975428104400635, -0.3618677854537964, -0.2126084268093109, -1.6099309921264648, -0.6661619544029236

In [ ]:
response

{'data': [{'embedding': [-1.436251163482666,
    -0.4631139636039734,
    -0.2887111008167267,
    0.1043136715888977,
    -0.18004582822322845,
    0.3143523633480072,
    -0.17387962341308594,
    0.6757763624191284,
    -1.3850526809692383,
    -0.3222927451133728,
    1.3471592664718628,
    0.33096542954444885,
    -0.7217952013015747,
    0.8920304775238037,
    -1.20805025100708,
    0.46127259731292725,
    -1.305800199508667,
    -0.8477014303207397,
    0.8280575275421143,
    0.2849760949611664,
    -1.4031113386154175,
    0.33332395553588867,
    -0.6530104875564575,
    -1.0358444452285767,
    0.4907127618789673,
    0.2932211756706238,
    0.6595003008842468,
    2.175729513168335,
    -0.5897372961044312,
    0.6383960247039795,
    -1.1273974180221558,
    -0.6744024157524109,
    1.082930326461792,
    1.0616108179092407,
    -0.07839717715978622,
    -1.8823751211166382,
    -0.028083711862564087,
    -0.17612075805664062,
    0.969205915927887,
    0.78446841239929

In [ ]:
len(response['data'][0]['embedding'])

96

In [ ]:
### We can even pass a list to get a list of embeddings for the texts

In [ ]:
import spacy
import numpy as np

# Load spaCy model with GloVe embeddings
import en_core_web_sm

nlp = en_core_web_sm.load()

def custom_embedding(text_list, model_name="text-embedding-ada-002"):
    embeddings = []

    for text in text_list:
        # Process the text with spaCy
        doc = nlp(text)

        # Extract word embeddings and average them to get the text embedding
        word_embeddings = [token.vector for token in doc if token.has_vector]

        if not word_embeddings:
            embeddings.append(None)  # No embeddings found for any word in the text
        else:
            text_embedding = np.mean(word_embeddings, axis=0)
            embeddings.append(text_embedding.tolist())

    # Create a response dictionary
    response = {
        "data": [
            {
                "embedding": emb,
                "index": idx,
                "object": "embedding"
            }
            for idx, emb in enumerate(embeddings)
        ],
        "model": model_name,
        "object": "list",
        "usage": {
            "prompt_tokens": sum(len(text.split()) for text in text_list),
            "total_tokens": sum(len(text.split()) for text in text_list)
        }
    }

    return response

# Example usage
text = ["She is running", "Fitness is good", "I am hungry", "Basketball is healthy"]
response = custom_embedding(text)

for idx, embedding in enumerate(response["data"]):
    if embedding["embedding"] is not None:
        print(f"Custom Embedding for '{text[idx]}': {embedding['embedding']}")
    else:
        print(f"No embeddings found for words in '{text[idx]}'.")

print(response)

Custom Embedding for 'She is running': [-0.7323653101921082, 0.29023632407188416, 0.4202596843242645, 0.6197046637535095, -0.2956758737564087, 0.005061978939920664, 0.9979009032249451, -0.6687696576118469, 0.3303249776363373, 0.9417006373405457, 1.0771689414978027, 0.0649028792977333, -0.4249298572540283, -0.02250763773918152, -0.5443629622459412, 0.4996660649776459, 0.312257319688797, -0.3544764518737793, -0.25966814160346985, 0.28132036328315735, -0.9640511870384216, 0.4155862033367157, 0.17327718436717987, -0.104308120906353, -0.3081322908401489, 0.03104131482541561, 0.043079983443021774, 0.5326375365257263, -0.2239867001771927, -0.21411271393299103, 0.1730899065732956, 0.5528287291526794, -0.36581850051879883, -0.5553879737854004, 0.25444722175598145, -0.1254696398973465, -0.29145944118499756, -0.8391502499580383, -0.000627537549007684, 0.8315939903259277, 0.7699801325798035, 0.2095264047384262, -0.2843492031097412, 0.09520425647497177, 0.721263587474823, -0.7334671020507812, 0.309

In [ ]:
print(response['data'][0]['embedding'])

[-0.7323653101921082, 0.29023632407188416, 0.4202596843242645, 0.6197046637535095, -0.2956758737564087, 0.005061978939920664, 0.9979009032249451, -0.6687696576118469, 0.3303249776363373, 0.9417006373405457, 1.0771689414978027, 0.0649028792977333, -0.4249298572540283, -0.02250763773918152, -0.5443629622459412, 0.4996660649776459, 0.312257319688797, -0.3544764518737793, -0.25966814160346985, 0.28132036328315735, -0.9640511870384216, 0.4155862033367157, 0.17327718436717987, -0.104308120906353, -0.3081322908401489, 0.03104131482541561, 0.043079983443021774, 0.5326375365257263, -0.2239867001771927, -0.21411271393299103, 0.1730899065732956, 0.5528287291526794, -0.36581850051879883, -0.5553879737854004, 0.25444722175598145, -0.1254696398973465, -0.29145944118499756, -0.8391502499580383, -0.000627537549007684, 0.8315939903259277, 0.7699801325798035, 0.2095264047384262, -0.2843492031097412, 0.09520425647497177, 0.721263587474823, -0.7334671020507812, 0.30969542264938354, 0.10631191730499268, 1.

In [ ]:
emb1 = response['data'][0]['embedding']
emb2 = response['data'][1]['embedding']
emb3 = response['data'][2]['embedding']
emb4 = response['data'][3]['embedding']

In [ ]:
## Text Similarity
np.dot(emb1, emb2)

12.806143513665408

In [ ]:
np.dot(emb2, emb4)

17.18689687926224

### Reading database and preprocessing

In [ ]:
df = pd.read_csv('/content/Dronealexa.csv')
df = df.dropna()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Title             5000 non-null   object
 1   URL               5000 non-null   object
 2   Publication Year  5000 non-null   int64 
 3   Abstract          5000 non-null   object
dtypes: int64(1), object(3)
memory usage: 156.4+ KB


In [ ]:
df.head()

,Title,URL,Publication Year,Abstract
0,AERIAL DRONE COMPANION DEVICE AND A METHOD OF ...,https://lens.org/059-472-052-640-920,2018,A method of operating an aerial drone companio...
1,Aerial drone companion device and a method of ...,https://lens.org/059-612-079-304-443,2018,A method of operating an aerial drone companio...
2,Onboard drone human-machine interface for auto...,https://lens.org/033-727-794-808-209,2021,An autonomous drone system uses an onboard com...
3,AUTONOMOUS NAVIGATION OF AN UNMANNED AERIAL VE...,https://lens.org/129-513-967-300-276,2018,A system for autonomous navigation of an unman...
4,"User equipment, system, and control method for...",https://lens.org/148-532-206-320-814,2023,Provided is a user equipment for controlling a...


[link text](https://)### Combining Questions and Answers field

In [ ]:
df['combined'] = "Title: " + df['Title'].str.strip() + "; URL: " + df['URL'].str.strip() + "; Publication Year: " + df['Publication Year'].astype(str).str.strip() + "; Abstract: " + df['Abstract'].str.strip()

In [ ]:
df.head()

,Title,URL,Publication Year,Abstract,combined
0,AERIAL DRONE COMPANION DEVICE AND A METHOD OF ...,https://lens.org/059-472-052-640-920,2018,A method of operating an aerial drone companio...,Title: AERIAL DRONE COMPANION DEVICE AND A MET...
1,Aerial drone companion device and a method of ...,https://lens.org/059-612-079-304-443,2018,A method of operating an aerial drone companio...,Title: Aerial drone companion device and a met...
2,Onboard drone human-machine interface for auto...,https://lens.org/033-727-794-808-209,2021,An autonomous drone system uses an onboard com...,Title: Onboard drone human-machine interface f...
3,AUTONOMOUS NAVIGATION OF AN UNMANNED AERIAL VE...,https://lens.org/129-513-967-300-276,2018,A system for autonomous navigation of an unman...,Title: AUTONOMOUS NAVIGATION OF AN UNMANNED AE...
4,"User equipment, system, and control method for...",https://lens.org/148-532-206-320-814,2023,Provided is a user equipment for controlling a...,"Title: User equipment, system, and control met..."


### Finding text lengths and bringing everything under 8000 tokens

In [ ]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
df['n_tokens'] = df.combined.progress_apply(lambda x: len(tokenizer.encode(x)))

  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
df = df[df.n_tokens < 8000]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Title             5000 non-null   object
 1   URL               5000 non-null   object
 2   Publication Year  5000 non-null   int64 
 3   Abstract          5000 non-null   object
 4   combined          5000 non-null   object
 5   n_tokens          5000 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 273.4+ KB


In [ ]:
df.head()

,Title,URL,Publication Year,Abstract,combined,n_tokens
0,AERIAL DRONE COMPANION DEVICE AND A METHOD OF ...,https://lens.org/059-472-052-640-920,2018,A method of operating an aerial drone companio...,Title: AERIAL DRONE COMPANION DEVICE AND A MET...,211
1,Aerial drone companion device and a method of ...,https://lens.org/059-612-079-304-443,2018,A method of operating an aerial drone companio...,Title: Aerial drone companion device and a met...,198
2,Onboard drone human-machine interface for auto...,https://lens.org/033-727-794-808-209,2021,An autonomous drone system uses an onboard com...,Title: Onboard drone human-machine interface f...,182
3,AUTONOMOUS NAVIGATION OF AN UNMANNED AERIAL VE...,https://lens.org/129-513-967-300-276,2018,A system for autonomous navigation of an unman...,Title: AUTONOMOUS NAVIGATION OF AN UNMANNED AE...,60
4,"User equipment, system, and control method for...",https://lens.org/148-532-206-320-814,2023,Provided is a user equipment for controlling a...,"Title: User equipment, system, and control met...",125


# Getting started with embeddings

### Defining basic function

In [ ]:
import spacy
import numpy as np

# Load spaCy model with GloVe embeddings
import en_core_web_sm

nlp = en_core_web_sm.load()

def get_embeddings(text, model):
    # Process the text with spaCy
    doc = model(text)

    # Extract word embeddings and average them to get the text embedding
    word_embeddings = [token.vector for token in doc if token.has_vector]

    if not word_embeddings:
        return None  # No embeddings found for any word in the text

    text_embedding = np.mean(word_embeddings, axis=0)

    # Create a response dictionary
    response = {
        "data": [
            {
                "embedding": text_embedding.tolist(),
                "index": 0,
                "object": "embedding"
            }
        ],
        "model": model.meta["name"],
        "object": "list",
        "usage": {
            "prompt_tokens": len(text.split()),
            "total_tokens": len(doc)
        }
    }

    return response

# Example usage
input_text = "Your input text goes here"
custom_model = nlp  # You can replace this with any other spaCy model

# Renaming 'input_text' to avoid conflict with the built-in 'input' function
text_to_process = input_text

response = get_embeddings(text_to_process, custom_model)

if response["data"][0]["embedding"] is not None:
    print(f"Custom Embedding for '{text_to_process}': {response['data'][0]['embedding']}")
else:
    print(f"No embeddings found for words in '{text_to_process}'.")

print(response)

Custom Embedding for 'Your input text goes here': [0.06477324664592743, -0.11796951293945312, 0.10780811309814453, -0.10981075465679169, -0.21296969056129456, -0.19421939551830292, -0.04632258415222168, 0.650249719619751, -0.08722390979528427, -0.41815271973609924, 0.23291008174419403, -0.00028890371322631836, -0.14824312925338745, -0.457096666097641, 0.06577853858470917, 0.16513226926326752, -0.2932334840297699, 0.1279958188533783, -0.038502298295497894, -0.037103213369846344, 0.23997125029563904, 0.45197200775146484, 0.10690854489803314, -0.25067731738090515, 0.3115770220756531, -0.13251809775829315, -0.2835305333137512, -0.10260505974292755, -0.18233594298362732, -0.12483687698841095, -0.1709585189819336, 0.10970769822597504, 0.20151738822460175, -1.1323931217193604, 0.31994733214378357, -0.4716111123561859, 0.31998711824417114, -0.19265961647033691, 0.4623194634914398, 0.5886304974555969, -0.5359376668930054, 0.5727989673614502, 0.5446508526802063, 0.3134579658508301, -0.1464684158

### Applying embeddings function on each row of the 'combined' column in batches of 2000

In [ ]:
from tqdm import tqdm

batch_size = 2000
model_name = 'text-embedding-ada-002'

# Assuming df is your DataFrame
for i in tqdm(range(0, len(df.combined), batch_size)):
    # find end of batch
    i_end = min(i + batch_size, len(df.combined))

    # Get embeddings for the current batch
    batch_text = list(df.combined)[i:i_end]

    # Initialize an empty list to store the embeddings for each text in the batch
    batch_embeddings = []

    # Process each text in the batch and get embeddings
    for text in batch_text:
        response = get_embeddings(text, nlp)

        # Check if embeddings were found
        if response and response["data"][0]["embedding"] is not None:
            batch_embeddings.append(response["data"][0]["embedding"])
        else:
            # Handle the case where no embeddings are found for a text
            batch_embeddings.append(None)

    # Update the DataFrame with the embeddings
    for j in range(i, i_end):
        df.loc[j, 'ada_vector'] = str(batch_embeddings[j - i])

100%|██████████| 3/3 [02:26<00:00, 48.98s/it]


In [ ]:
df.head()

,Title,URL,Publication Year,Abstract,combined,n_tokens,ada_vector
0,AERIAL DRONE COMPANION DEVICE AND A METHOD OF ...,https://lens.org/059-472-052-640-920,2018,A method of operating an aerial drone companio...,Title: AERIAL DRONE COMPANION DEVICE AND A MET...,211,"[0.2901443839073181, -0.3325439691543579, 0.06..."
1,Aerial drone companion device and a method of ...,https://lens.org/059-612-079-304-443,2018,A method of operating an aerial drone companio...,Title: Aerial drone companion device and a met...,198,"[0.2968430519104004, -0.32281967997550964, 0.0..."
2,Onboard drone human-machine interface for auto...,https://lens.org/033-727-794-808-209,2021,An autonomous drone system uses an onboard com...,Title: Onboard drone human-machine interface f...,182,"[0.11472707986831665, -0.24680840969085693, -0..."
3,AUTONOMOUS NAVIGATION OF AN UNMANNED AERIAL VE...,https://lens.org/129-513-967-300-276,2018,A system for autonomous navigation of an unman...,Title: AUTONOMOUS NAVIGATION OF AN UNMANNED AE...,60,"[0.10273130238056183, -0.24615001678466797, -0..."
4,"User equipment, system, and control method for...",https://lens.org/148-532-206-320-814,2023,Provided is a user equipment for controlling a...,"Title: User equipment, system, and control met...",125,"[0.0363294780254364, -0.1987609565258026, -0.0..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Title             5000 non-null   object
 1   URL               5000 non-null   object
 2   Publication Year  5000 non-null   int64 
 3   Abstract          5000 non-null   object
 4   combined          5000 non-null   object
 5   n_tokens          5000 non-null   int64 
 6   ada_vector        5000 non-null   object
dtypes: int64(2), object(5)
memory usage: 441.5+ KB


### Converting datatype of embeddings to a vector

In [ ]:
df['ada_vector'] = df.ada_vector.progress_apply(eval).progress_apply(np.array)

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

### Saving the DataFrame to csv for future access

In [ ]:
df.to_csv('embeddings_chatbot.csv',index=False)

In [ ]:
df=pd.read_csv('embeddings_chatbot.csv')

## Creating query

In [ ]:
user_query = input("Enter query - ")

query_response = get_embeddings(user_query, nlp)

if query_response["data"][0]["embedding"] is not None:
    print(f"Embedding for '{user_query}': {query_response['data'][0]['embedding']}")
else:
    print(f"No embeddings found for words in '{user_query}'.")

Enter query - AERIAL DRONE COMPANION DEVICE AND A METHOD OF OPERATING AN AERIAL DRONE COMPANION DEVICE
Embedding for 'AERIAL DRONE COMPANION DEVICE AND A METHOD OF OPERATING AN AERIAL DRONE COMPANION DEVICE': [0.11798430234193802, -0.2902727425098419, -0.15090186893939972, 0.23681941628456116, -0.20924879610538483, 0.03403221815824509, 0.30266842246055603, 0.2975979149341583, 0.28998130559921265, 0.08998402208089828, 0.04189266636967659, 0.5342325568199158, -0.7161141037940979, -0.6303009390830994, 0.12806786596775055, 0.21904683113098145, -0.6586694717407227, -0.14464303851127625, -0.28388240933418274, -0.11181040853261948, -0.12043266743421555, 0.04661577567458153, 0.35065627098083496, -0.2839812636375427, 0.5602681040763855, -0.3193376362323761, 0.5237229466438293, 0.17741039395332336, 0.7041237950325012, 0.18627846240997314, -0.14661884307861328, -0.5036998391151428, -0.07061945647001266, 0.14741185307502747, -0.24235336482524872, 0.06453554332256317, 0.15608465671539307, 0.2936713

In [ ]:
query_response

{'data': [{'embedding': [0.11798430234193802,
    -0.2902727425098419,
    -0.15090186893939972,
    0.23681941628456116,
    -0.20924879610538483,
    0.03403221815824509,
    0.30266842246055603,
    0.2975979149341583,
    0.28998130559921265,
    0.08998402208089828,
    0.04189266636967659,
    0.5342325568199158,
    -0.7161141037940979,
    -0.6303009390830994,
    0.12806786596775055,
    0.21904683113098145,
    -0.6586694717407227,
    -0.14464303851127625,
    -0.28388240933418274,
    -0.11181040853261948,
    -0.12043266743421555,
    0.04661577567458153,
    0.35065627098083496,
    -0.2839812636375427,
    0.5602681040763855,
    -0.3193376362323761,
    0.5237229466438293,
    0.17741039395332336,
    0.7041237950325012,
    0.18627846240997314,
    -0.14661884307861328,
    -0.5036998391151428,
    -0.07061945647001266,
    0.14741185307502747,
    -0.24235336482524872,
    0.06453554332256317,
    0.15608465671539307,
    0.2936713993549347,
    0.1829686015844345,
  

In [ ]:
query_response['data'][0]['embedding']

[0.11798430234193802,
 -0.2902727425098419,
 -0.15090186893939972,
 0.23681941628456116,
 -0.20924879610538483,
 0.03403221815824509,
 0.30266842246055603,
 0.2975979149341583,
 0.28998130559921265,
 0.08998402208089828,
 0.04189266636967659,
 0.5342325568199158,
 -0.7161141037940979,
 -0.6303009390830994,
 0.12806786596775055,
 0.21904683113098145,
 -0.6586694717407227,
 -0.14464303851127625,
 -0.28388240933418274,
 -0.11181040853261948,
 -0.12043266743421555,
 0.04661577567458153,
 0.35065627098083496,
 -0.2839812636375427,
 0.5602681040763855,
 -0.3193376362323761,
 0.5237229466438293,
 0.17741039395332336,
 0.7041237950325012,
 0.18627846240997314,
 -0.14661884307861328,
 -0.5036998391151428,
 -0.07061945647001266,
 0.14741185307502747,
 -0.24235336482524872,
 0.06453554332256317,
 0.15608465671539307,
 0.2936713993549347,
 0.1829686015844345,
 0.0021870818454772234,
 -0.6220111846923828,
 0.5763250589370728,
 0.00551415141671896,
 0.7437050938606262,
 -0.06620777398347855,
 -0.433

In [ ]:
searchvector = get_embeddings(user_query, custom_model)["data"][0]["embedding"]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Assuming df['ada_vector'] contains the vectors you want to compare

# Ensure 'ada_vector' column contains valid numeric arrays
df['ada_vector'] = df['ada_vector'].apply(lambda x: np.array(x) if isinstance(x, (list, np.ndarray)) else x)

# Filter out rows where 'ada_vector' is not a valid numeric array
valid_rows = df['ada_vector'].apply(lambda x: isinstance(x, np.ndarray))

# Calculate cosine similarity only for valid rows
df.loc[valid_rows, 'similarities'] = df.loc[valid_rows, 'ada_vector'].apply(
    lambda x: cosine_similarity([x], [searchvector])[0][0]
)

# If you are using the 'progress_apply' from the 'tqdm' library
# You can keep it as follows:
# df.loc[valid_rows, 'similarities'] = df.loc[valid_rows, 'ada_vector'].progress_apply(
#     lambda x: cosine_similarity([x], [searchvector])[0][0]
# )


In [ ]:
df.head()

,Title,URL,Publication Year,Abstract,combined,n_tokens,ada_vector,similarities
0,AERIAL DRONE COMPANION DEVICE AND A METHOD OF ...,https://lens.org/059-472-052-640-920,2018,A method of operating an aerial drone companio...,Title: AERIAL DRONE COMPANION DEVICE AND A MET...,211,[ 0.29014438 -0.33254397 0.06361313 0.077943...,NaN
1,Aerial drone companion device and a method of ...,https://lens.org/059-612-079-304-443,2018,A method of operating an aerial drone companio...,Title: Aerial drone companion device and a met...,198,[ 0.29684305 -0.32281968 0.06620878 0.064001...,NaN
2,Onboard drone human-machine interface for auto...,https://lens.org/033-727-794-808-209,2021,An autonomous drone system uses an onboard com...,Title: Onboard drone human-machine interface f...,182,[ 0.11472708 -0.24680841 -0.08849844 -0.021271...,NaN
3,AUTONOMOUS NAVIGATION OF AN UNMANNED AERIAL VE...,https://lens.org/129-513-967-300-276,2018,A system for autonomous navigation of an unman...,Title: AUTONOMOUS NAVIGATION OF AN UNMANNED AE...,60,[ 0.1027313 -0.24615002 -0.16022387 -0.099052...,NaN
4,"User equipment, system, and control method for...",https://lens.org/148-532-206-320-814,2023,Provided is a user equipment for controlling a...,"Title: User equipment, system, and control met...",125,[ 0.03632948 -0.19876096 -0.07848036 -0.002820...,NaN


In [ ]:
df.sort_values('similarities', ascending = False)

,Title,URL,Publication Year,Abstract,combined,n_tokens,ada_vector,similarities
0,AERIAL DRONE COMPANION DEVICE AND A METHOD OF ...,https://lens.org/059-472-052-640-920,2018,A method of operating an aerial drone companio...,Title: AERIAL DRONE COMPANION DEVICE AND A MET...,211,[ 0.29014438 -0.33254397 0.06361313 0.077943...,NaN
1,Aerial drone companion device and a method of ...,https://lens.org/059-612-079-304-443,2018,A method of operating an aerial drone companio...,Title: Aerial drone companion device and a met...,198,[ 0.29684305 -0.32281968 0.06620878 0.064001...,NaN
2,Onboard drone human-machine interface for auto...,https://lens.org/033-727-794-808-209,2021,An autonomous drone system uses an onboard com...,Title: Onboard drone human-machine interface f...,182,[ 0.11472708 -0.24680841 -0.08849844 -0.021271...,NaN
3,AUTONOMOUS NAVIGATION OF AN UNMANNED AERIAL VE...,https://lens.org/129-513-967-300-276,2018,A system for autonomous navigation of an unman...,Title: AUTONOMOUS NAVIGATION OF AN UNMANNED AE...,60,[ 0.1027313 -0.24615002 -0.16022387 -0.099052...,NaN
4,"User equipment, system, and control method for...",https://lens.org/148-532-206-320-814,2023,Provided is a user equipment for controlling a...,"Title: User equipment, system, and control met...",125,[ 0.03632948 -0.19876096 -0.07848036 -0.002820...,NaN
...,...,...,...,...,...,...,...,...
4995,MOBILE INDUSTRIAL ROBOT WITH SECURITY SYSTEM F...,https://lens.org/012-366-442-461-341,2020,There is provided a mobile industrial robot wi...,Title: MOBILE INDUSTRIAL ROBOT WITH SECURITY S...,100,[ 1.84643686e-01 -6.50210679e-02 5.84785230e-...,NaN
4996,MOBILE INDUSTRIAL ROBOT WITH SECURITY SYSTEM F...,https://lens.org/012-366-442-461-341,2020,There is provided a mobile industrial robot wi...,Title: MOBILE INDUSTRIAL ROBOT WITH SECURITY S...,100,[ 1.84643686e-01 -6.50210679e-02 5.84785230e-...,NaN
4997,First response locator system,https://lens.org/081-559-374-227-193,2018,A first response locator system includes at le...,Title: First response locator system; URL: htt...,134,[ 0.18112426 -0.14445643 -0.14316188 0.081026...,NaN
4998,FIRST RESPONSE LOCATOR SYSTEM,https://lens.org/077-920-075-664-684,2017,A first response locator system includes at le...,Title: FIRST RESPONSE LOCATOR SYSTEM; URL: htt...,137,[ 0.18139781 -0.14681913 -0.12517869 0.094854...,NaN


In [ ]:
result = df.sort_values('similarities', ascending = False).head(3)

In [ ]:
result.head()

,Title,URL,Publication Year,Abstract,combined,n_tokens,ada_vector,similarities
0,AERIAL DRONE COMPANION DEVICE AND A METHOD OF ...,https://lens.org/059-472-052-640-920,2018,A method of operating an aerial drone companio...,Title: AERIAL DRONE COMPANION DEVICE AND A MET...,211,[ 0.29014438 -0.33254397 0.06361313 0.077943...,NaN
1,Aerial drone companion device and a method of ...,https://lens.org/059-612-079-304-443,2018,A method of operating an aerial drone companio...,Title: Aerial drone companion device and a met...,198,[ 0.29684305 -0.32281968 0.06620878 0.064001...,NaN
2,Onboard drone human-machine interface for auto...,https://lens.org/033-727-794-808-209,2021,An autonomous drone system uses an onboard com...,Title: Onboard drone human-machine interface f...,182,[ 0.11472708 -0.24680841 -0.08849844 -0.021271...,NaN


In [ ]:
xc = list(result.combined)

## Constructing prompt for GPT


In [ ]:
def construct_prompt(query, xc):
  context = ''
  for i in range(3):
    context += xc[i] + "\n"
  header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
  header += context + "\n\n Q: " + query + "\n A:"
  return header


[link text](https://)### Let's see how constructed prompt looks like

In [ ]:
# Result=print(construct_prompt(user_query, xc))

Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."

Context:
Title: AERIAL DRONE COMPANION DEVICE AND A METHOD OF OPERATING AN AERIAL DRONE COMPANION DEVICE; URL: https://lens.org/059-472-052-640-920; Publication Year: 2018; Abstract: A method of operating an aerial drone companion device includes detecting a first voice command spoken by a first user. The aerial drone companion device is autonomously oriented such that an image capture device faces the first user in response to detecting the first voice command. A second voice command spoken by the first user is detected while the image capture device faces the first user. The second voice command is transmitted from the aerial drone companion device to a computer located remotely from the aerial drone companion device. A task signal is received indicating a task to be performed. The task signal is generated by the computer based on th

In [ ]:
# !pip install transformers

In [ ]:
from transformers import pipeline

In [ ]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
# Replace this line
# summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False)

# With this line
Fresult = construct_prompt(user_query, xc)
summarizer("\n".join(xc), max_length=130, min_length=30, do_sample=False)
# print("\nGenerated Summary:\n", summary)


[{'summary_text': ' A method of operating an aerial drone companion device includes detecting a first voice command spoken by a first user . A second voice command is detected while the image capture device faces the first user in response to detecting the first command . A task signal is received indicating a task to be performed . The method includes autonomously executing the task by the aerial drone .'}]